# Simple models for the observation operator

Before applying the complex LSTM model, we'll first check the performance of much simpler ML models: linear regression and ridge regression.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from pathlib import Path

pad = Path(os.getcwd())
if pad.name == "ml_observation_operator":
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.PDM import PDM

%load_ext autoreload 
%autoreload 2 

In [ ]:
os.getcwd()

## Load in data

In [ ]:
# PDM
#Forcing data
preprocess_output_folder = Path('data/Zwalm_data/preprocess_output')
p_zwalm = pd.read_pickle(preprocess_output_folder/'zwalm_p_thiessen.pkl')
ep_zwalm = pd.read_pickle(preprocess_output_folder/'zwalm_ep_thiessen.pkl')

#Parameterset
param = pd.read_csv("data/Zwalm_PDM_parameters/p1_opt_param_mNSE_PSO_70_particles_qconst_strict.csv", index_col = False)
param = param.drop(param.columns[0], axis = 1)
param

#Area Zwalm
zwalm_shape = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma_31370.shp')
area_zwalm_new = np.single(zwalm_shape.area[0]/10**6)

In [ ]:
#Features for data-driven model
features = pd.read_csv('data/g0_OpenEO/s1_g0_timeseries.csv', parse_dates=True)
features = features.set_index('t')
features.index = pd.to_datetime(features.index)
print(features.columns)
cols = features.columns[features.columns.str.endswith(('Pasture','Agriculture','Forest'))]
#Make dummy variables of the orbit direction
orb_dir_dummies = pd.get_dummies(features.Orbitdirection)
orb_dir_dummies = orb_dir_dummies.set_index(features.index)
features = pd.concat([features[cols],orb_dir_dummies],axis = 1)
features.head()

In [ ]:
features.index

In [ ]:
features_corr = features.corr()
features_corr.style.background_gradient(cmap = 'coolwarm')

Run the PDM for the desired parameterset

In [ ]:
deltat = np.array(1,dtype = np.float32) #hour
deltat_out = np.array(24, dtype = np.float32) #daily averaging
pd_zwalm_out_day = PDM(P = p_zwalm['P_thiessen'].values, 
    EP = ep_zwalm['EP_thiessen'].values,
    t = p_zwalm['Timestamp'].values,
    area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,
    parameters = param, m = 3)
pd_zwalm_out_day = pd_zwalm_out_day.set_index('Time')

In [ ]:
pd_zwalm_out_day['Cstar'].plot(title = 'C*', ylabel = '[mm]')

In [ ]:
pd_zwalm_out_day 

In [ ]:
Cstar = pd_zwalm_out_day['Cstar']
Cstar.head()

## Split up dataset in training and validation part

In [ ]:
t1_features = features.index[0]
print('day 1 SAR data: ' + str(t1_features))
tend_features = features.index[-1]
print('last day SAR data: ' + str(tend_features))
nr_days = tend_features - t1_features
nr_years = nr_days.total_seconds()/(3600*24*365.25)
print('years of SAR data: ' + str(nr_years))

Take 5.5 years of training data and the remaining 2 as validation data

In [ ]:
tend_calibration = pd.Timestamp(datetime(year = 2020, month = 12, day = 31))
tbegin_validation = pd.Timestamp(datetime(year = 2021, month = 1, day = 1))

print('Calibration period: ' + str(t1_features) + ' until ' + str(tend_calibration))
print('Validation period: ' + str(tbegin_validation) + ' until ' + str(tend_features))


In [ ]:
X_train = features[t1_features:tend_calibration]
X_test = features[tbegin_validation:tend_features]
 
#select only on days with available training data! 
y_train = Cstar[X_train.index]
y_test = Cstar[X_test.index]

## Linear regression

Idea of linear regression as observation operator alreayd applied in Auber(cf. [obsidian](C:\Users\olivi\Documents\ob_obsidian\DA\Aubert_SM_DA_in_conceptual_model.md) and https://www.sciencedirect.com/science/article/pii/S0022169403002294?via%3Dihub )

Include forest in the equation

https://en.wikipedia.org/wiki/Coefficient_of_determination#Adjusted_R2 

Calculated adjusted $R^2$ (=$\bar{R}^2$) from regular $R^2$ as:
$$
{\displaystyle {\bar {R}}^{2}=1-(1-R^{2}){n-1 \over n-p}}
$$

with $n$ the number of variables  and $p$ the number parameters 

In [ ]:
LinReg = LinearRegression()
LinReg.fit(X_train, y_train)
y_train_hat = LinReg.predict(X_train)

#Also try normalising both input and output before prediction
LinRegnorm = make_pipeline(StandardScaler(), LinearRegression())
scaler_y = StandardScaler()
scaler_y.fit(y_train.values.reshape(-1, 1))
y_train_norm = scaler_y.transform(y_train.values.reshape(-1, 1))
y_test_norm = scaler_y.transform(y_test.values.reshape(-1, 1))
LinRegnorm.fit(X_train, y_train_norm)
y_train_norm_hat = LinRegnorm.predict(X_train)
y_test_norm_hat = LinRegnorm.predict(X_test)

#Trainig set performance
mse_train = mean_squared_error(y_train, y_train_hat)
mse_train_norm = mean_squared_error(y_train, scaler_y.inverse_transform(y_train_norm_hat))
r2_train = r2_score(y_train, y_train_hat)
r2_train_norm = r2_score(y_train,scaler_y.inverse_transform(y_train_norm_hat))
n_train = X_train.shape[0]
p = LinReg.coef_.shape[0] + 1 #+1 for the intercept
r2_adjsted_train = 1 - (1-r2_train)*(n_train - 1)/(n_train - p)
r2_adjsted_train_norm = 1 - (1-r2_train_norm)*(n_train - 1)/(n_train - p)
print('RMSE on training data: ' + str(np.sqrt(mse_train)))
print('RMSE on training data with normalisation: ' + str(np.sqrt(mse_train_norm)))
print('R2 on training data: ' + str(r2_train))
print('R2 on training data with normalisation: ' + str(r2_train_norm))
print('Adjusted R2 on training data: ' + str(r2_adjsted_train))
print('Adjusted R2 on training data with normalisation: ' + str(r2_adjsted_train_norm))



Not sure how to calculate R2 adjusted on test data

In [ ]:
# Test set performance
y_test_hat = LinReg.predict(X_test)
y_test_norm_hat = LinRegnorm.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_hat)
mse_test_norm = mean_squared_error(y_test, scaler_y.inverse_transform(y_test_norm_hat))
r2_test = r2_score(y_test, y_test_hat)
r2_test_norm = r2_score(y_test, scaler_y.inverse_transform(y_test_norm_hat))
print('RMSE on test data: ' + str(np.sqrt(mse_test)))
print('RMSE on test data with normalisation: ' + str(np.sqrt(mse_test_norm)))
print('R2 on test data: ' + str(r2_test))
print('R2 on test data with normalisation: ' + str(r2_test_norm))

Conclusion on normalisation: not really necessary in this case, basically no difference in performance! 

In [ ]:
coef_dict =  {}
for i, param in enumerate(X_train.columns.to_list()):
    coef_dict[param] = LinReg.coef_[i]
pd_coef = pd.DataFrame(coef_dict, index =[0])
pd_coef

Visualise the predictions

In [ ]:
Cstar_SAR_time = pd_zwalm_out_day.loc[features.index, 'Cstar']
fig,ax = plt.subplots()
Cstar_SAR_time.plot(ax = ax, ylabel = 'C* [mm]', label = 'PDM')
plt.plot(X_train.index, y_train_hat, label = 'Train', alpha = 0.7)
plt.plot(X_test.index, y_test_hat, label = 'Test', alpha = 0.7)
ax.legend()
ax.set_title('Linear regression')

## Ridge regression

L2 normalisation. Well explained in https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html automatic best hyperparamter $\alpha$ (often called $\lambda$) for regularisation by appyling crosss validation. cv = 5 is 5-fold cross-validation within the training set

In [ ]:
ridge = RidgeCV(alphas = np.logspace(-3,3,100), cv = 5)
ridge.fit(X_train, y_train)
y_train_hat_ridge = ridge.predict(X_train)

#Trainig set performance
mse_train_ridge = mean_squared_error(y_train, y_train_hat_ridge)
r2_train_ridge = r2_score(y_train, y_train_hat_ridge)
n_train = X_train.shape[0]
p = ridge.coef_.shape[0] + 1 #+1 for the intercept
r2_adjsted_train_ridge = 1 - (1-r2_train_ridge)*(n_train - 1)/(n_train - p)
print('RMSE on training data: ' + str(np.sqrt(mse_train_ridge)))
print('R2 on training data: ' + str(r2_train))
print('Adjusted R2 on training data: ' + str(r2_adjsted_train_ridge))

In [ ]:
# Test set performance
y_test_hat_ridge = ridge.predict(X_test)
mse_test_ridge = mean_squared_error(y_test, y_test_hat_ridge)
r2_test_ridge = r2_score(y_test, y_test_hat_ridge)
print('RMSE on test data ridge regression: ' + str(np.sqrt(mse_test_ridge)))
print('R2 on test data ridge regression: ' + str(r2_test_ridge))
print('RMSE on test data linear regression: ' + str(np.sqrt(mse_test)))
print('R2 on test data linear regression: ' + str(r2_test))

Very minimal differences with normal linear regression! So very little regularisation needed!

In [ ]:
fig,ax = plt.subplots()
Cstar_SAR_time.plot(ax = ax, ylabel = 'C* [mm]', label = 'PDM')
plt.plot(X_train.index, y_train_hat_ridge, label = 'Train', alpha = 0.7)
plt.plot(X_test.index, y_test_hat_ridge, label = 'Test', alpha = 0.7)
ax.legend()
ax.set_title('Ridge regression')

Leaving out all forest data lead to significantly worse performance on preliminary test (mainly due to less degrees of freedom I believe)

## Support Vector regression

https://scikit-learn.org/stable/modules/svm.html#svm-regression


C and epsilon to be optimised => cross validation ideally

In [ ]:
svr_nonlin = make_pipeline(StandardScaler(), SVR(kernel = 'rbf', C = 100, epsilon = 0.2))
svr_nonlin.fit(X_train, y_train)
y_train_SVR = svr_nonlin.predict(X_train)
r2_SVR = r2_score(y_train, y_train_SVR)
print('trainig score SVR: ' + str(r2_SVR))

y_test_SVR = svr_nonlin.predict(X_test)
r2_SVR_test = r2_score(y_test, y_test_SVR)
print('trainig score SVR: ' + str(r2_SVR_test))

In [ ]:
fig,ax = plt.subplots()
Cstar_SAR_time.plot(ax = ax, ylabel = 'C* [mm]', label = 'PDM')
plt.plot(X_train.index, y_train_SVR, label = 'Train', alpha = 0.7)
plt.plot(X_test.index, y_test_SVR, label = 'Test', alpha = 0.7)
ax.legend()
ax.set_title('SVR')